## This notebook might not be worked: due to changing of modules

In [1]:
import torch
import torchvision
import shutil 

batch_size= 32

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('../datasets/mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                #torchvision.transforms.Normalize(
                                # (0.5,), (0.5,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../datasets/mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                #torchvision.transforms.Normalize(
                                # (0.5,), (0.5,))
                             ])),
  batch_size=16, shuffle=False, drop_last= True)

In [2]:
import torch
import shutil 
import numpy as np
from torch import nn
from modules.kernels import get_gaussian
from modules.models.decoder import simple_generator
from modules.models.forward_model import forward_modelA
from modules.models.forward_H import modelH
from modules.train_utils_v2 import train
from modules.custom_activations import inc_m
import matplotlib.pyplot as plt
from modules.models.preprocess_H_weights import ifft, fft
from modules.noise import poisson_noise
from modules.models.classifiers import simple_mnist_classifier

device='cuda' if torch.cuda.is_available() else 'cpu'

classifier = simple_mnist_classifier(32).to(device)
classifier.load_state_dict(torch.load('saved_models/mnist_classifier.pth', map_location=device)['model_state_dict'])

<All keys matched successfully>

In [3]:
img_size= 32
m_inc_epoc= 1

def inc_m_2(m, epoch):return inc_m(m, epoch, 2)

sPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)
exPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)

criterion= nn.L1Loss().to(device)
train_model_iter, train_H_iter= 1, 1
epochs=100
show_results_epoch=10

## Test run

In [ ]:
T=5
H_complex_init =False
H_weight_preprocess=  None
m_inc_proc =inc_m

decoder= simple_generator(T, img_size).to(device)
opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

H_generator = modelH(T, 32, H_weight_preprocess, H_complex_init, device).to(device)
opt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.001)

noise= poisson_noise(0.026, device)
train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, epochs, 1, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, None, noise, classifier)


## Experiments

In [5]:
#!rm -rf figs/mnist
#!mkdir figs/mnist

In [ ]:
Ts=[2, 3, 5, 16]
H_complex_inits = [True, False]
H_weight_preprocesses=  [None, fft, ifft, torch.abs]
m_inc_procs =  [inc_m, None]
poisson_rates= [0.001, 0.01, 0.02, 0.03, 0.05] #0.1, 0.5, 1.0]

for H_complex_init in H_complex_inits:
    for T in Ts:
        for m_inc_proc in m_inc_procs:
            #if  H_complex_init==True and T==2 and m_inc_proc==None:continue
            for H_weight_preprocess in H_weight_preprocesses:
                if H_weight_preprocess==None and H_complex_init==True:continue
                for rate in poisson_rates:
                    #if m_inc_proc==None and H_weight_preprocess==fft:continue
                    #if m_inc_proc==None and H_weight_preprocess==ifft and rate in [0.001, 0.01, 0.02]:continue
                    try:
                        
                        if H_weight_preprocess ==None:H_weight_preprocess_name = 'None'
                        else:H_weight_preprocess_name = H_weight_preprocess.__name__
                        if m_inc_proc==None:m_inc_proc_name = 'None'
                        else:m_inc_proc_name = m_inc_proc.__name__
                        
                        save_dir= f'figs/mnist/m_inc_proc({m_inc_proc_name})@T({T})@H_complex_init({H_complex_init})@H_weight_preprocess({H_weight_preprocess_name})@poisson_rate({rate})'
                        print(f'\n\nRUNNING EXPERIMENT :: {save_dir}')
                        
                        try:shutil.rmtree(save_dir)
                        except:pass
                        
                        
                        decoder= simple_generator(T, img_size).to(device)
                        opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

                        H_generator = modelH(T, 32, H_weight_preprocess, H_complex_init, device).to(device)
                        opt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.001)

                        noise= poisson_noise(rate, device)
                        train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, epochs, show_results_epoch, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, save_dir, noise, classifier)

                    except:
                        print(f'ERROR :: figs/mnist/m_inc_proc({m_inc_proc_name})@T({T})@H_complex_init({H_complex_init})@H_weight_preprocess({H_weight_preprocess_name})@poisson_rate({rate})')
                        pass



RUNNING EXPERIMENT :: figs/mnist/m_inc_proc(inc_m)@T(16)@H_complex_init(True)@H_weight_preprocess(fft)@poisson_rate(0.001)
device : cuda
m : 1
